In [1]:
import sys
print(sys.version)
print(sys.executable) #path of the Python executable being used by Jupyter Notebook.

3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)]
C:\ProgramData\Anaconda3\envs\tf\python.exe


In [2]:
# 1. Define the problem
## Predict resale HDB property prices

In [3]:
# 2. Pre-process data
## Collect data
## Clean data
## Wrangle data

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df_hdb = pd.read_csv("data.csv")

In [6]:
df_hdb.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [7]:
# Check dtype of features
df_hdb.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154668 entries, 0 to 154667
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                154668 non-null  object 
 1   town                 154668 non-null  object 
 2   flat_type            154668 non-null  object 
 3   block                154668 non-null  object 
 4   street_name          154668 non-null  object 
 5   storey_range         154668 non-null  object 
 6   floor_area_sqm       154668 non-null  float64
 7   flat_model           154668 non-null  object 
 8   lease_commence_date  154668 non-null  int64  
 9   remaining_lease      154668 non-null  object 
 10  resale_price         154668 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 13.0+ MB


In [8]:
# Check missing values of features
df_hdb.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
remaining_lease        0
resale_price           0
dtype: int64

In [9]:
# Convert 'remaining_lease' in string to 'remaining_lease_months' in numerical

In [10]:
# Function to convert string to months
def convert_to_months(duration_str):
    split_parts = duration_str.split(' ')
    
    if len(split_parts) == 4:
        years = int(split_parts[0])
        months = int(split_parts[2])
        total_months = years * 12 + months
    
    else:
        years = int(split_parts[0])
        total_months = years * 12
        
    return total_months

# Apply the conversion function to the Duration column
df_hdb['remaining_lease_months']  = df_hdb['remaining_lease'].apply(convert_to_months)

In [11]:
#drop remaining_lease
df_hdb.drop('remaining_lease', axis=1, inplace=True)

In [12]:
df_hdb.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease_months
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,232000.0,736
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,250000.0,727
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,262000.0,749
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,265000.0,745
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,265000.0,749


In [13]:
X_df_hdb = df_hdb[['remaining_lease_months',
                    'floor_area_sqm',
                    'town',
                    'storey_range']]
X_df_hdb

,remaining_lease_months,floor_area_sqm,town,storey_range
0,736,44.0,ANG MO KIO,10 TO 12
1,727,67.0,ANG MO KIO,01 TO 03
2,749,67.0,ANG MO KIO,01 TO 03
3,745,68.0,ANG MO KIO,04 TO 06
4,749,67.0,ANG MO KIO,01 TO 03
...,...,...,...,...
154663,732,84.0,YISHUN,04 TO 06
154664,753,83.0,YISHUN,10 TO 12
154665,737,84.0,YISHUN,04 TO 06
154666,761,104.0,YISHUN,04 TO 06


In [14]:
y_df_hdb = df_hdb['resale_price']
y_df_hdb

0         232000.0
1         250000.0
2         262000.0
3         265000.0
4         265000.0
            ...   
154663    410000.0
154664    455000.0
154665    458000.0
154666    575000.0
154667    715000.0
Name: resale_price, Length: 154668, dtype: float64

In [15]:
# Encode categorical columns

In [16]:
categorical_columns = X_df_hdb.select_dtypes(include='object').columns.tolist()
categorical_columns

['town', 'storey_range']

In [17]:
X_df_hdb['town'].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'PUNGGOL',
       'QUEENSTOWN', 'SEMBAWANG', 'SENGKANG', 'SERANGOON', 'TAMPINES',
       'TOA PAYOH', 'WOODLANDS', 'YISHUN'], dtype=object)

In [18]:
town_mapping = {
    'ANG MO KIO': 0,
    'BEDOK': 1,
    'BISHAN': 2,
    'BUKIT BATOK': 3,
    'BUKIT MERAH': 4,
    'BUKIT PANJANG': 5,
    'BUKIT TIMAH': 6,
    'CENTRAL AREA': 7,
    'CHOA CHU KANG': 8,
    'CLEMENTI': 9,
    'GEYLANG': 10,
    'HOUGANG': 11,
    'JURONG EAST': 12,
    'JURONG WEST': 13,
    'KALLANG/WHAMPOA': 14,
    'MARINE PARADE': 15,
    'PASIR RIS': 16,
    'PUNGGOL': 17,
    'QUEENSTOWN': 18,
    'SEMBAWANG': 19,
    'SENGKANG': 20,
    'SERANGOON': 21,
    'TAMPINES': 22,
    'TOA PAYOH': 23,
    'WOODLANDS': 24,
    'YISHUN': 25
}


In [19]:
storey_mapping = {
    '01 TO 03': 0,
    '04 TO 06': 1,
    '07 TO 09': 2,
    '10 TO 12': 3,
    '13 TO 15': 4,
    '16 TO 18': 5,
    '19 TO 21': 6,
    '22 TO 24': 7,
    '25 TO 27': 8,
    '28 TO 30': 9,
    '31 TO 33': 10,
    '34 TO 36': 11,
    '37 TO 39': 12,
    '40 TO 42': 13,
    '43 TO 45': 14,
    '46 TO 48': 15,
    '49 TO 51': 16
}


In [20]:
X_df_hdb['town'] = X_df_hdb['town'].map(town_mapping)
X_df_hdb['storey_range'] = X_df_hdb['storey_range'].map(storey_mapping)
X_df_hdb.head()

C:\Users\Zach Chen\AppData\Local\Temp\ipykernel_4060\398897315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df_hdb['town'] = X_df_hdb['town'].map(town_mapping)
C:\Users\Zach Chen\AppData\Local\Temp\ipykernel_4060\398897315.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_df_hdb['storey_range'] = X_df_hdb['storey_range'].map(storey_mapping)


,remaining_lease_months,floor_area_sqm,town,storey_range
0,736,44.0,0,3
1,727,67.0,0,0
2,749,67.0,0,0
3,745,68.0,0,1
4,749,67.0,0,0


In [21]:
# Scale all features

In [22]:
# from sklearn.preprocessing import MinMaxScaler
# import joblib

# scaler = MinMaxScaler()
# X_df_hdb_scaled_array = scaler.fit_transform(X_df_hdb)

# joblib.dump(scaler, 'scaler.joblib')


In [23]:
def scale_features(df):
    scaled_df = pd.DataFrame()
    for column in df.columns:
        max_value = df[column].max()
        min_value = df[column].min()
        scaled_df[column] = (df[column] - min_value) / (max_value - min_value)
    return scaled_df

In [24]:
X_df_hdb_scaled = scale_features(X_df_hdb)
X_df_hdb_scaled.head()

,remaining_lease_months,floor_area_sqm,town,storey_range
0,0.339879,0.059633,0.0,0.1875
1,0.326284,0.165138,0.0,0.0000
2,0.359517,0.165138,0.0,0.0000
3,0.353474,0.169725,0.0,0.0625
4,0.359517,0.165138,0.0,0.0000


In [25]:
# X_df_hdb_scaled = pd.DataFrame(X_df_hdb_scaled_array, columns = X_df_hdb.columns)
# X_df_hdb_scaled.head()

In [26]:
# 5. Modelling Data

In [27]:
## Split into train and test sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df_hdb_scaled, y_df_hdb, test_size = 0.3, random_state=43)

In [28]:
X_train.head()

,remaining_lease_months,floor_area_sqm,town,storey_range
136781,0.342900,0.284404,0.96,0.0000
81017,0.451662,0.527523,0.32,0.1250
140608,0.329305,0.197248,0.88,0.0625
67301,0.901813,0.288991,0.80,0.1250
109494,0.889728,0.165138,0.80,0.3750


In [29]:
y_train.head()

136781    408000.0
81017     550000.0
140608    390000.0
67301     425000.0
109494    419000.0
Name: resale_price, dtype: float64

In [30]:
# using XGBoost regression

In [31]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
import math

In [32]:
# initialise params grid
max_depth_list = [9, 12, 15]
n_estimators_list = [100, 200, 300, 400, 500]

# initialise evaluation grid
R2_score_test= []
RMSE_score_test= []
R2_score_train= []
RMSE_score_train= []


for i in n_estimators_list:
    
    for j in max_depth_list:
        params = {'objective': 'reg:squarederror', 'n_estimators': i, 'max_depth': j}
        model = XGBRegressor(**params)

        # fit the train data set to the model:

        model.fit(X_train, y_train)

        y_pred= model.predict(X_test)
        y_pred_train= model.predict(X_train)

        coefficient_of_dermination_test = r2_score(y_test, y_pred)
        coefficient_of_dermination_train = r2_score(y_train, y_pred_train)
        
        R2_score_test.append(coefficient_of_dermination_test)
        R2_score_train.append(coefficient_of_dermination_train)
    
        MSE_test = mean_squared_error(y_test, y_pred)  
        RMSE_test = math.sqrt(MSE_test)
        RMSE_score_test.append(RMSE_test)
    
        MSE_train = mean_squared_error(y_train, y_pred_train)  
        RMSE_train = math.sqrt(MSE_train)
        RMSE_score_train.append(RMSE_train)

In [33]:
xgb_df= pd.DataFrame({'max_depth': [9,9,9,9,9,12,12,12,12,12,15,15,15,15,15], 
                      'n_estimators': [100, 200, 300, 400, 500, 100, 200, 300, 400, 500, 100, 200, 300, 400, 500],
                      'R2_test': R2_score_test, 
                      'RMSE_test': RMSE_score_test,
                      'R2_train': R2_score_train,
                      'RMSE_train': RMSE_score_train})
xgb_df

,max_depth,n_estimators,R2_test,RMSE_test,R2_train,RMSE_train
0,9,100,0.905016,51226.490072,0.934721,42742.681172
1,9,200,0.902779,51826.033816,0.962241,32507.527002
2,9,300,0.893306,54292.468764,0.977613,25030.342669
3,9,400,0.905208,51174.711320,0.948942,37801.112298
4,9,500,0.897128,53311.149376,0.973868,27043.504525
5,12,100,0.885553,56230.326439,0.983194,21687.611320
6,12,200,0.903519,51628.631844,0.957071,34661.537222
7,12,300,0.892226,54566.540055,0.979258,24093.368737
8,12,400,0.881425,57235.572404,0.984495,20830.654412
9,12,500,0.901606,52137.919787,0.962452,32416.467873


In [34]:
best_xgb= {'model': 'XGBoost Regression', 
           'R2_test': max(R2_score_test),
           'RMSE_test': float(xgb_df[xgb_df['R2_test'] == max(R2_score_test)]['RMSE_test']),
           'Parameter': f"max_depth= {float(xgb_df[xgb_df['R2_test'] == max(R2_score_test)]['max_depth'])}, n_estimators= {float(xgb_df[xgb_df['R2_test'] == max(R2_score_test)]['n_estimators'])}",
           'R2_train': float(xgb_df[xgb_df['R2_test'] == max(R2_score_test)]['R2_train'])}
best_xgb

{'model': 'XGBoost Regression',
 'R2_test': 0.9052076189455107,
 'RMSE_test': 51174.71132009564,
 'Parameter': 'max_depth= 9.0, n_estimators= 400.0',
 'R2_train': 0.9489422305825752}

In [35]:
#Instantiate xgb_model obj using XGBRegressor() class
params = {'objective': 'reg:squarederror', 'n_estimators': 400, 'max_depth': 9}
xgb_model = XGBRegressor(**params) #**params unpacks values from the dictionary and passed them in as keyword arguments to the XGBRegressor constructor

In [ ]:
xgb_model.fit(X_train, y_train)

In [42]:
# deploy ridge_model

In [43]:
import joblib
joblib.dump(xgb_model, 'xgb_model.pkl')

['xgb_model.pkl']